In [ ]:
from google.colab import files, drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -r ./drive/MyDrive/Hate_detection/Hinglish_Hate_Detection/requirements.txt

     |████████████████████████████████| 81kB 7.0MB/s 
     |████████████████████████████████| 516.2MB 24kB/s 
     |████████████████████████████████| 573kB 41.5MB/s 
     |████████████████████████████████| 686kB 45.9MB/s 
     |████████████████████████████████| 1.9MB 44.7MB/s 
     |████████████████████████████████| 3.0MB 45.0MB/s 
     |████████████████████████████████| 512kB 48.2MB/s 
     |████████████████████████████████| 1.7MB 46.6MB/s 
     |████████████████████████████████| 389kB 46.3MB/s 
     |████████████████████████████████| 358kB 57.2MB/s 
     |████████████████████████████████| 1.6MB 44.7MB/s 
     |████████████████████████████████| 5.7MB 46.3MB/s 
     |████████████████████████████████| 112kB 54.4MB/s 
     |████████████████████████████████| 890kB 53.1MB/s 
     |████████████████████████████████| 71kB 10.5MB/s 
     |████████████████████████████████| 51kB 475kB/s 
     |████████████████████████████████| 460kB 55.1MB/s 
     |████████████████████████████████| 133kB 56.7MB/

In [ ]:

from __future__ import absolute_import

import sys
import os

sys.path.append('./drive/MyDrive/Hate_detection/Hinglish_Hate_Detection/')


In [ ]:
#!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
from __future__ import absolute_import

import sys
import os

import shutil

try:
    from dotenv import find_dotenv, load_dotenv
except:
    pass

import argparse

try:
    sys.path.append(os.path.join(os.path.dirname(__file__), './drive/MyDrive/Hate_detection/Hinglish_Hate_Detection/'))
except:
    sys.path.append(os.path.join(os.getcwd(), './drive/MyDrive/Hate_detection/Hinglish_Hate_Detection/'))
    
try:
    sys.path.append(os.path.join(os.path.dirname(__file__), '../'))
except:
    sys.path.append(os.path.join(os.getcwd(), '../'))
    
import pandas as pd
import numpy as np

import pickle
from collections import Counter
from tqdm import tqdm

import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow_addons as tfa

try:
    import wandb
    load_dotenv(find_dotenv())
    wandb.login(key=os.environ['WANDB_API_KEY'])
    from wandb.keras import WandbCallback
    _has_wandb = True
except:
    _has_wandb = False

import tokenizers
from transformers import TFAutoModel, AutoTokenizer, AutoConfig, BertTokenizer

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer

from src import data, models

pd.options.display.max_colwidth = -1

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: WARNING Calling wandb.login() without arguments from jupyter should prompt you for an api key.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/victor/.netrc


In [ ]:
print (_has_wandb)

In [ ]:
parser = argparse.ArgumentParser(prog='Trainer',conflict_handler='resolve')

#parser.add_argument('--train_data', type=str, default='./drive/MyDrive/Hate_detection/Hinglish_Hate_Detection/data/raw/Hate-speech-dataset/hate_speech.tsv', required=False,
#                    help='train data')
parser.add_argument('--train_data', type=str, default='../data/raw/Hate-speech-dataset/hate_speech.tsv', required=False,
                    help='train data')

parser.add_argument('--val_data', type=str, default=None, required=False,
                    help='validation data')
parser.add_argument('--test_data', type=str, default=None, required=False,
                    help='test data')

parser.add_argument('--transformer_model_pretrained_path', type=str, default='roberta-base', required=False,
                    help='transformer model pretrained path or huggingface model name')
parser.add_argument('--transformer_config_path', type=str, default='roberta-base', required=False,
                    help='transformer config file path or huggingface model name')
parser.add_argument('--transformer_tokenizer_path', type=str, default='roberta-base', required=False,
                    help='transformer tokenizer file path or huggingface model name')

parser.add_argument('--max_text_len', type=int, default=50, required=False,
                    help='maximum length of text')
parser.add_argument('--max_char_len', type=int, default=200, required=False,
                    help='maximum length of text')
parser.add_argument('--max_word_char_len', type=int, default=20, required=False,
                    help='maximum length of text')

parser.add_argument('--emb_dim', type=int, default=128, required=False,
                    help='maximum length of text')
parser.add_argument('--n_layers', type=int, default=2, required=False,
                    help='maximum length of text')
parser.add_argument('--n_units', type=int, default=128, required=False,
                    help='maximum length of text')

parser.add_argument('--epochs', type=int, default=500, required=False,
                    help='number of epochs')
parser.add_argument('--lr', type=float, default=.001, required=False,
                    help='learning rate')
parser.add_argument('--early_stopping_rounds', type=int, default=50, required=False,
                    help='number of epochs for early stopping')
parser.add_argument('--lr_schedule_round', type=int, default=30, required=False,
                    help='number of epochs for learning rate scheduling')

parser.add_argument('--train_batch_size', type=int, default=32, required=False,
                    help='train batch size')
parser.add_argument('--eval_batch_size', type=int, default=16, required=False,
                    help='eval batch size')

#parser.add_argument('--model_save_path', type=str, default='./drive/MyDrive/Hate_detection/Hinglish_Hate_Detection/models/model_hindi_sentiment/', required=False,
#                    help='seed')

parser.add_argument('--model_save_path', type=str, default='../models/hate_detection/', required=False,
                    help='seed')

parser.add_argument('--wandb_logging', type=bool, default=True, required=False,
                    help='wandb logging needed')

parser.add_argument('--seed', type=int, default=42, required=False,
                    help='seed')


args, _ = parser.parse_known_args()

In [ ]:
tf.random.set_seed(args.seed)
np.random.seed(args.seed)

In [ ]:
df = pd.read_csv('./drive/MyDrive/Hate_detection/Hinglish_Hate_Detection/data/raw/Hate-speech-dataset/hate_speech.tsv', \
                 sep='\t',header=None,usecols=[0,1])
df.columns = ['text','category']
df = df.dropna()
df = df[df.text != '']

kf = KFold(n_splits=5, shuffle=True, random_state=args.seed)
for train_index, test_index in kf.split(df.text):
    break

df['type'] = 'hate'

hate_train_df = df.iloc[train_index]
kf2 = KFold(n_splits=2, shuffle=True, random_state=args.seed)
for val_index, test_index in kf2.split(df.iloc[test_index].text):
    break

hate_val_df = df.iloc[val_index]
hate_test_df = df.iloc[test_index]

df = pd.read_csv('./drive/MyDrive/Hate_detection/Hinglish_Hate_Detection/data/processed/StanceDetection_CodeMixed/data.txt', \
                 sep='\t',header=None,usecols=[0,1])
df.columns = ['text','category']
df = df.dropna()
df = df[df.text != '']

kf = KFold(n_splits=5, shuffle=True, random_state=args.seed)
for train_index, test_index in kf.split(df.text):
    break

df['type'] = 'stance'
stance_train_df = df.iloc[train_index]
kf2 = KFold(n_splits=2, shuffle=True, random_state=args.seed)
for val_index, test_index in kf2.split(df.iloc[test_index].text):
    break

stance_val_df = df.iloc[val_index]
stance_test_df = df.iloc[test_index]

df = pd.read_csv('./drive/MyDrive/Hate_detection/Hinglish_Hate_Detection/data/processed/SarcasmDetection_CodeMixed/data.txt', \
                 sep='\t',header=None,usecols=[0,1])
df.columns = ['text','category']
df = df.dropna()
df = df[df.text != '']

kf = KFold(n_splits=5, shuffle=True, random_state=args.seed)
for train_index, test_index in kf.split(df.text):
    break

df['type'] = 'sarcasm'

sarcasm_train_df = df.iloc[train_index]
kf2 = KFold(n_splits=2, shuffle=True, random_state=args.seed)
for val_index, test_index in kf2.split(df.iloc[test_index].text):
    break

sarcasm_val_df = df.iloc[val_index]
sarcasm_test_df = df.iloc[test_index]

df = pd.read_csv('./drive/MyDrive/Hate_detection/Hinglish_Hate_Detection/data/processed/humor-detection-corpus/data.txt', \
                 sep='\t',header=None,usecols=[0,1])
df.columns = ['text','category']
df = df.dropna()
df = df[df.text != '']

kf = KFold(n_splits=5, shuffle=True, random_state=args.seed)
for train_index, test_index in kf.split(df.text):
    break

df['type'] = 'humor'

humor_train_df = df.iloc[train_index]
kf2 = KFold(n_splits=2, shuffle=True, random_state=args.seed)
for val_index, test_index in kf2.split(df.iloc[test_index].text):
    break

humor_val_df = df.iloc[val_index]
humor_test_df = df.iloc[test_index]

df = pd.read_csv('./drive/MyDrive/Hate_detection/Hinglish_Hate_Detection/data/processed/Aggression_dataset/train.txt', \
                 sep='\t',header=None,usecols=[0,1])
df = pd.concat([df,pd.read_csv('./drive/MyDrive/Hate_detection/Hinglish_Hate_Detection/data/processed/Aggression_dataset/val.txt', \
                 sep='\t',header=None,usecols=[0,1])],axis=0)

df.columns = ['text','category']
df = df.dropna()
df = df[df.text != '']

kf = KFold(n_splits=5, shuffle=True, random_state=args.seed)
for train_index, test_index in kf.split(df.text):
    break

aggression_train_df = df.iloc[train_index]
kf2 = KFold(n_splits=2, shuffle=True, random_state=args.seed)
for val_index, test_index in kf2.split(df.iloc[test_index].text):
    break

aggression_val_df = df.iloc[val_index]
aggression_test_df = df.iloc[test_index]


In [ ]:
train_df = pd.concat([hate_train_df,hate_train_df,sarcasm_train_df,humor_train_df,aggression_train_df],axis=0)
val_df = pd.concat([hate_val_df,hate_val_df,sarcasm_val_df,humor_val_df,aggression_val_df],axis=0)
val_df = pd.concat([hate_test_df,hate_test_df,sarcasm_test_df,humor_test_df,aggression_test_df],axis=0)

print (train_df.shape, val_df.shape, test_df.shape)

(3662, 2) (458, 2) (458, 2)


In [ ]:
train_df.head(5)

,text,category
0,"Knowing ki Vikas kitna samjhata hai Priyanka aur Itch Guard Luv ko, usne bola tha Ben wali baat me ab Sallu ne bhi agree kiya!",no
1,I am Muhajir .. Aur mere lye sab se Pehly Pakistan he .. agr 10 lakh Altaf Jese leaders bh is zameen ki behurmati kren un sbko sar e aam phansi Deni chahye .. Proud to be a #Muhajir and #Pakistani,no
2,"Doctor sab sahi me ke PhD (in hate politics) wale. Bhai padhe likhe ho fir kyu ye sab baate karte ho. Tum bas bowling khelo, aur maje lo. pic.twitter.com/fk1qUbQstw",no
3,"Poore Desh me Patel OBC me aate Hain sirf gujrat Ko chor kar may be, ye manuwadiyon bramanwadi kabhi aapko aarackchan nahi denge ye to jis OBC Ko Mila hai usse bhi nafrat karte hain ye khoon aur chamdi ka frak karne waale bharmhanwadi kisi ke sage nahi hain",no
4,"Sarkar banne ke bad Hindu hit me ek bhi faisla Jo bjp ke dwara liya gaya ho,bjp ko gay,gobar,mandir,masjid aur nafrat faila kar vot chahiye",yes


In [ ]:
train_df.text = train_df.text.apply(lambda x: data.preprocessing.clean_tweets(x))
val_df.text = val_df.text.apply(lambda x: data.preprocessing.clean_tweets(x))
test_df.text = test_df.text.apply(lambda x: data.preprocessing.clean_tweets(x))

train_df = train_df[train_df.text != '']
val_df = val_df[val_df.text != '']
test_df = test_df[test_df.text != '']

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
hate_train_df.text = hate_train_df.text.apply(lambda x: data.preprocessing.clean_tweets(x))
hate_val_df.text = hate_val_df.text.apply(lambda x: data.preprocessing.clean_tweets(x))
hate_test_df.text = hate_test_df.text.apply(lambda x: data.preprocessing.clean_tweets(x))

hate_train_df = hate_train_df[hate_train_df.text != '']
hate_val_df = hate_val_df[hate_val_df.text != '']
hate_test_df = hate_test_df[hate_test_df.text != '']


In [ ]:
train_df.text.apply(lambda x: len(x)).describe()

count    3661.000000
mean     102.369571 
std      53.175251  
min      6.000000   
25%      63.000000  
50%      99.000000  
75%      130.000000 
max      313.000000 
Name: text, dtype: float64

In [ ]:
train_df.text.apply(lambda x: len(x.split())).describe()

count    3661.000000
mean     19.663207  
std      10.279373  
min      1.000000   
25%      12.000000  
50%      19.000000  
75%      25.000000  
max      64.000000  
Name: text, dtype: float64

In [ ]:
model_save_dir = args.model_save_path

try:
    os.makedirs(model_save_dir)
except OSError:
    pass

In [ ]:
hate_train_df.category.value_counts()

no     2316
yes    1342
on     2   
n      1   
Name: category, dtype: int64

In [ ]:
train_df = train_df[train_df.category.str.contains('yes|no')]
val_df = val_df[val_df.category.str.contains('yes|no')]
test_df = test_df[test_df.category.str.contains('yes|no')]


In [ ]:
hate_train_df = hate_train_df[hate_train_df.category.str.contains('yes|no')]
hate_val_df = hate_val_df[hate_val_df.category.str.contains('yes|no')]
hate_test_df = hate_test_df[hate_test_df.category.str.contains('yes|no')]


In [ ]:
model_save_dir

'./drive/MyDrive/Hate_detection/Hinglish_Hate_Detection/models/model_hindi_sentiment/'

In [ ]:
hate_train_df.category, label2idx = data.data_utils.convert_categorical_label_to_int(hate_train_df.category.values, \
                                                         save_path=os.path.join(model_save_dir,'label2idx.pkl'))

hate_val_df.category, _ = data.data_utils.convert_categorical_label_to_int(hate_val_df.category.values, \
                                                         save_path=os.path.join(model_save_dir,'label2idx.pkl'))

hate_test_df.category, _ = data.data_utils.convert_categorical_label_to_int(hate_test_df.category.values, \
                                                         save_path=os.path.join(model_save_dir,'label2idx.pkl'))

In [ ]:
train_df.head(5)

,text,category
0,"knowing ki vikas kitna samjhata hai priyanka aur itch guard luv ko, usne bola tha ben wali baat me ab sallu ne bhi agree kiya!",0
1,i am muhajir .. aur mere lye sab se pehly pakistan he .. agr lakh altaf jese leaders bh is zameen ki behurmati kren un sbko sar e aam phansi deni chahye .. proud to be a and,0
2,"doctor sab sahi me ke phd (in hate politics) wale. bhai padhe likhe ho fir kyu ye sab baate karte ho. tum bas bowling khelo, aur maje lo. pic.twitter.com/fkqubqstw",0
3,"poore desh me patel obc me aate hain sirf gujrat ko chor kar may be, ye manuwadiyon bramanwadi kabhi aapko aarackchan nahi denge ye to jis obc ko mila hai usse bhi nafrat karte hain ye khoon aur chamdi ka frak karne waale bharmhanwadi kisi ke sage nahi hain",0
4,"sarkar banne ke bad hindu hit me ek bhi faisla jo bjp ke dwara liya gaya ho,bjp ko gay,gobar,mandir,masjid aur nafrat faila kar vot chahiye",1


In [ ]:
print (label2idx)

{'no': 0, 'yes': 1}


In [ ]:
idx2label = {i:w for (w,i) in label2idx.items()}

### Learn tokenizer

In [ ]:
data.custom_tokenizers.custom_wp_tokenizer(hate_train_df.text.values, args.model_save_path, args.model_save_path)
tokenizer = BertTokenizer.from_pretrained(args.model_save_path)

In [ ]:
word_tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=50000, split=' ',oov_token=1)
char_tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True, split='',oov_token=1)

word_tokenizer.fit_on_texts(hate_train_df.text.values)
char_tokenizer.fit_on_texts(hate_train_df.text.values)

In [ ]:
transformer_train_inputs, _, _ = data.data_utils.compute_transformer_input_arrays(train_df, 'text', tokenizer, args.max_char_len)

word_train_inputs = word_tokenizer.texts_to_sequences(train_df.text.values)
word_train_inputs = tf.keras.preprocessing.sequence.pad_sequences(word_train_inputs, maxlen=args.max_text_len)

subword_train_inputs = np.asarray([data.data_utils.subword_tokenization(text, char_tokenizer, args.max_text_len, args.max_word_char_len) \
                        for text in tqdm(train_df.text.values)])

char_train_inputs = char_tokenizer.texts_to_sequences(train_df.text.values)
char_train_inputs = tf.keras.preprocessing.sequence.pad_sequences(char_train_inputs, maxlen=args.max_char_len)

train_outputs = data.data_utils.compute_output_arrays(train_df, 'category')

transformer_val_inputs, _, _ = data.data_utils.compute_transformer_input_arrays(hate_val_df, 'text', tokenizer, args.max_char_len)

word_val_inputs = word_tokenizer.texts_to_sequences(hate_val_df.text.values)
word_val_inputs = tf.keras.preprocessing.sequence.pad_sequences(word_val_inputs, maxlen=args.max_text_len)

subword_val_inputs = np.asarray([data.data_utils.subword_tokenization(text, char_tokenizer, args.max_text_len, args.max_word_char_len) \
                        for text in tqdm(hate_val_df.text.values)])

char_val_inputs = char_tokenizer.texts_to_sequences(hate_val_df.text.values)
char_val_inputs = tf.keras.preprocessing.sequence.pad_sequences(char_val_inputs, maxlen=args.max_char_len)

val_outputs = data.data_utils.compute_output_arrays(hate_val_df, 'category')

transformer_test_inputs, _, _ = data.data_utils.compute_transformer_input_arrays(test_df, 'text', tokenizer, args.max_char_len)

word_test_inputs = word_tokenizer.texts_to_sequences(test_df.text.values)
word_test_inputs = tf.keras.preprocessing.sequence.pad_sequences(word_test_inputs, maxlen=args.max_text_len)

subword_test_inputs = np.asarray([data.data_utils.subword_tokenization(text, char_tokenizer, args.max_text_len, args.max_word_char_len) \
                        for text in tqdm(test_df.text.values)])

char_test_inputs = char_tokenizer.texts_to_sequences(test_df.text.values)
char_test_inputs = tf.keras.preprocessing.sequence.pad_sequences(char_test_inputs, maxlen=args.max_char_len)

test_outputs = data.data_utils.compute_output_arrays(test_df, 'category')

train_outputs = tf.keras.utils.to_categorical(train_outputs, \
                                                    num_classes=train_df.category.nunique())
val_outputs = tf.keras.utils.to_categorical(val_outputs, \
                                                    num_classes=train_df.category.nunique())
test_outputs = tf.keras.utils.to_categorical(test_outputs, \
                                                    num_classes=train_df.category.nunique())

print (transformer_train_inputs.shape, subword_train_inputs.shape, word_train_inputs.shape, char_train_inputs.shape, \
       train_outputs.shape)
print (transformer_val_inputs.shape, subword_val_inputs.shape, word_val_inputs.shape, char_val_inputs.shape, \
       val_outputs.shape)
print (transformer_test_inputs.shape, subword_test_inputs.shape, word_test_inputs.shape, char_test_inputs.shape, \
       test_outputs.shape)


100%|██████████| 458/458 [00:00<00:00, 1387.32it/s]


(3658, 200) (3658, 50, 20) (3658, 50) (3658, 200) (3658, 18201) (3658, 2)
(457, 200) (457, 50, 20) (457, 50) (457, 200) (457, 18201) (457, 2)
(458, 200) (458, 50, 20) (458, 50) (458, 200) (458, 18201) (458, 2)


### Modeling

In [ ]:
n_words = len(word_tokenizer.word_index)+1
n_chars = len(char_tokenizer.word_index)+1
n_subwords = tokenizer.vocab_size
tfidf_shape = train_tfidf.shape[1]
n_out = train_df.category.nunique()

In [ ]:
from src.models.models import *

#all_models = {HAN.__name__: HAN, \
#              CMSA.__name__: CMSA, CS_ELMO_without_words.__name__: CS_ELMO_without_words}

all_models = {HAN.__name__: HAN}

In [ ]:
_has_wandb = False

In [ ]:
if os.path.exists(os.path.join(args.model_save_path,'results.csv')):
  results = pd.read_csv(os.path.join(args.model_save_path,'results.csv'))
  index = results.shape[0]
  print (results)
else:
  results = pd.DataFrame(columns=['config','weighted_f1','macro_f1'])
  index = 0

for model_name, model_ in all_models.items():
    
    for loss in ['ce','focal']:
        
        for use_features in [True, False]:
            
            if use_features == False:
                model = model_(word_vocab_size=n_words,char_vocab_size=n_chars,wpe_vocab_size=n_subwords, n_out=n_out,max_word_char_len=args.max_word_char_len,\
                                             max_text_len=args.max_text_len, max_char_len=args.max_char_len,\
                                             n_layers=args.n_layers, n_units=args.n_units, emb_dim=args.emb_dim)
            else:
                model = model_(word_vocab_size=n_words,char_vocab_size=n_chars,wpe_vocab_size=n_subwords, n_out=n_out,vectorizer_shape=tfidf_shape, max_word_char_len=args.max_word_char_len,\
                                             max_text_len=args.max_text_len, max_char_len=args.max_char_len,\
                                             n_layers=args.n_layers, n_units=args.n_units, emb_dim=args.emb_dim)
            
            if use_features == True:
                print ("Running {} with features for {} loss".format(model_name, loss))
            else:
                print ("Running {} without features for {} loss".format(model_name, loss))

            print (model.summary())

            if loss == 'focal':
                model.compile(loss=models.utils.categorical_focal_loss(alpha=1), optimizer='adam', metrics=['accuracy', models.utils.f1_keras]) #binary_crossentropy
            elif loss == 'ce':
                model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', models.utils.f1_keras]) 

            lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.7, \
                                                  patience=args.lr_schedule_round, verbose=1, mode='auto', min_lr=0.000001)
            config = {
                  'text_max_len': args.max_text_len,
                  'char_max_len': args.max_char_len,
                  'word_char_max_len': args.max_word_char_len,
                  'n_units': args.n_units,
                  'emb_dim': args.emb_dim,
                  'n_layers': args.n_layers,
                  'epochs': args.epochs,
                  "learning_rate": args.lr,
                  "model_name": model_name,
                  "loss": loss,
                  "use_features": use_features
                }

            if use_features == True:
                model_save_path = os.path.join(args.model_save_path, '{}_{}_with_features.h5'.format(model_name, config['loss']))
            else:
                model_save_path = os.path.join(args.model_save_path, '{}_{}_without_features.h5'.format(model_name, config['loss']))

            f1callback = models.utils.F1Callback(model, [word_val_inputs, char_val_inputs, subword_val_inputs, transformer_val_inputs, val_tfidf],\
                                      val_outputs, \
                                      filename=model_save_path, \
                                      patience=args.early_stopping_rounds)

            K.clear_session()

            if _has_wandb and args.wandb_logging:
                wandb.init(project='hate_speech_detection',config=config)
                model.fit([word_train_inputs, char_train_inputs, subword_train_inputs, transformer_train_inputs, train_tfidf], train_outputs, \
                      validation_data=([word_val_inputs, char_val_inputs, subword_val_inputs, transformer_val_inputs, val_tfidf], val_outputs), \
                          epochs=args.epochs,batch_size=args.train_batch_size, callbacks=[lr, f1callback, WandbCallback()], verbose=2)
            else:
                model.fit([word_train_inputs, char_train_inputs, subword_train_inputs, transformer_train_inputs, train_tfidf], train_outputs, \
                      validation_data=([word_val_inputs, char_val_inputs, subword_val_inputs, transformer_val_inputs, val_tfidf], val_outputs), \
                          epochs=args.epochs,batch_size=args.train_batch_size, callbacks=[lr, f1callback], verbose=2)


            model.load_weights(model_save_path)

            test_pred = model.predict([word_test_inputs, char_test_inputs, subword_test_inputs, transformer_test_inputs, test_tfidf])

            report = classification_report([idx2label[i] for i in test_outputs.argmax(-1)], \
                                           [idx2label[i] for i in test_pred.argmax(-1)])

            f1 = f1_score([idx2label[i] for i in test_outputs.argmax(-1)], \
                                           [idx2label[i] for i in test_pred.argmax(-1)], average='weighted')

            print (report, f1)
            
            results.loc[index,'config'] = str(config)
            results.loc[index, 'weighted_f1'] = f1_score([idx2label[i] for i in test_outputs.argmax(-1)], \
                                           [idx2label[i] for i in test_pred.argmax(-1)], average='weighted')
            results.loc[index, 'macro_f1'] = f1_score([idx2label[i] for i in test_outputs.argmax(-1)], \
                                           [idx2label[i] for i in test_pred.argmax(-1)], average='macro')
            
            index += 1
            
            results.to_csv(os.path.join(args.model_save_path,'results.csv'),index=False)

                                                                                                                                                                                                                                     config  ...  macro_f1
0   {'text_max_len': 50, 'char_max_len': 200, 'word_char_max_len': 20, 'n_units': 128, 'emb_dim': 128, 'n_layers': 2, 'epochs': 500, 'learning_rate': 0.001, 'model_name': 'CS_ELMO_without_words', 'loss': 'ce', 'use_features': False}     ...  0.938759
1   {'text_max_len': 50, 'char_max_len': 200, 'word_char_max_len': 20, 'n_units': 128, 'emb_dim': 128, 'n_layers': 2, 'epochs': 500, 'learning_rate': 0.001, 'model_name': 'CS_ELMO_without_words', 'loss': 'focal', 'use_features': False}  ...  0.918659
2   {'text_max_len': 50, 'char_max_len': 200, 'word_char_max_len': 20, 'n_units': 128, 'emb_dim': 128, 'n_layers': 2, 'epochs': 500, 'learning_rate': 0.001, 'model_name': 'HAN', 'loss': 'ce', 'use_features': False}                       ...  0.921

In [ ]:
results

,config,weighted_f1,macro_f1
0,"{'text_max_len': 50, 'char_max_len': 200, 'word_char_max_len': 20, 'n_units': 128, 'emb_dim': 128, 'n_layers': 2, 'epochs': 500, 'learning_rate': 0.001, 'model_name': 'CS_ELMO_without_words', 'loss': 'ce', 'use_features': False}",0.947801,0.938759
1,"{'text_max_len': 50, 'char_max_len': 200, 'word_char_max_len': 20, 'n_units': 128, 'emb_dim': 128, 'n_layers': 2, 'epochs': 500, 'learning_rate': 0.001, 'model_name': 'CS_ELMO_without_words', 'loss': 'focal', 'use_features': False}",0.930531,0.918659
2,"{'text_max_len': 50, 'char_max_len': 200, 'word_char_max_len': 20, 'n_units': 128, 'emb_dim': 128, 'n_layers': 2, 'epochs': 500, 'learning_rate': 0.001, 'model_name': 'HAN', 'loss': 'ce', 'use_features': False}",0.932763,0.921351
3,"{'text_max_len': 50, 'char_max_len': 200, 'word_char_max_len': 20, 'n_units': 128, 'emb_dim': 128, 'n_layers': 2, 'epochs': 500, 'learning_rate': 0.001, 'model_name': 'HAN', 'loss': 'focal', 'use_features': False}",0.941107,0.930697
4,"{'text_max_len': 50, 'char_max_len': 200, 'word_char_max_len': 20, 'n_units': 128, 'emb_dim': 128, 'n_layers': 2, 'epochs': 500, 'learning_rate': 0.001, 'model_name': 'CMSA', 'loss': 'ce', 'use_features': False}",0.932640,0.921050
5,"{'text_max_len': 50, 'char_max_len': 200, 'word_char_max_len': 20, 'n_units': 128, 'emb_dim': 128, 'n_layers': 2, 'epochs': 500, 'learning_rate': 0.001, 'model_name': 'CMSA', 'loss': 'focal', 'use_features': False}",0.938865,0.927987
6,"{'text_max_len': 50, 'char_max_len': 100, 'word_char_max_len': 20, 'n_units': 128, 'emb_dim': 128, 'n_layers': 2, 'epochs': 500, 'learning_rate': 0.001, 'model_name': 'HIT', 'loss': 'ce', 'use_features': True}",0.931815,0.919118
7,"{'text_max_len': 50, 'char_max_len': 100, 'word_char_max_len': 20, 'n_units': 128, 'emb_dim': 128, 'n_layers': 2, 'epochs': 500, 'learning_rate': 0.001, 'model_name': 'HIT', 'loss': 'ce', 'use_features': False}",0.924222,0.911536
8,"{'text_max_len': 50, 'char_max_len': 100, 'word_char_max_len': 20, 'n_units': 128, 'emb_dim': 128, 'n_layers': 2, 'epochs': 500, 'learning_rate': 0.001, 'model_name': 'HIT', 'loss': 'focal', 'use_features': True}",0.929537,0.916332
9,"{'text_max_len': 50, 'char_max_len': 100, 'word_char_max_len': 20, 'n_units': 128, 'emb_dim': 128, 'n_layers': 2, 'epochs': 500, 'learning_rate': 0.001, 'model_name': 'HIT', 'loss': 'focal', 'use_features': False}",0.931963,0.919455


In [ ]:
results.to_csv(os.path.join(args.model_save_path,'results.csv'),index=False)